In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from urllib.request import urlretrieve
import re
from datetime import datetime

url = 'https://entertain.naver.com/ranking'
driver = webdriver.Chrome()
driver.get(url)

# posts 변수를 함수 외부에서 정의
posts = []

# 페이지 스크래핑 함수 정의
def scrape_page(driver):
    post_list = driver.find_elements(By.XPATH, '//*[@id="ranking_list"]/li')

    for i in post_list:
        post = i.find_element(By.XPATH, 'div[2]/a')
        news_title = post.text

        link = post.get_attribute('href')

        try:
            # 이미지 요소 찾기
            news_image = i.find_element(By.XPATH, 'a/img')
            news_image_url = news_image.get_attribute('src')

            # 정규표현식으로 숫자만 추출하여 파일 이름 생성
            match = re.search(r'(\d+)\.jpg', news_image_url)
            if match:
                news_image_name = match.group() 
            else:
                news_image_name = ''  # 추출 실패 시 빈 문자열로 설정

            print(news_image_name)

            news_image = news_image_url.replace('https', 'http')
            news_image_path = f'C:/Users/user/Downloads/news_image/entertain_image/{news_image_name}'
            urlretrieve(news_image, news_image_path)
        except:
            news_image_path = ''

        driver.execute_script(f"window.open('{link}', '_blank');")
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)

        create_dt = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[2]/span/em').text

        input_data = create_dt.replace("오후", "PM").replace("오전", "AM")
        create_dt = datetime.strptime(input_data, "%Y.%m.%d. %p %I:%M")
        print(create_dt)

        news_name = driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div/div[1]/a/img').get_attribute('alt')
        print(news_name)

        post_origin_text = driver.find_element(By.XPATH, '//*[@id="articeBody"]').text

        posts.append([create_dt, news_name, 'Entertain', news_image_path, news_title, post_origin_text, link])

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

# 첫 페이지 스크래핑
scrape_page(driver)

# 페이지 이동 및 스크래핑
for page_number in range(1, 7):  
    next_page_xpath = f'//*[@id="newsWrp"]/div[3]/a[{page_number}]'
    try:
        next_page_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, next_page_xpath))
        )
        next_page_button.click()
        time.sleep(2)  # 페이지 전환이 완료될 때까지 기다립니다.
        scrape_page(driver)
    except Exception as e:
        print(f"다음 페이지 이동 중 오류 발생: {e}")

7233319.jpg
2023-12-13 19:07:00
뉴스1
1671802.jpg
2023-12-13 20:14:00
엑스포츠뉴스
464461.jpg
2023-12-13 22:11:00
스포티비뉴스
1671838.jpg
2023-12-13 22:31:00
엑스포츠뉴스
464457.jpg
2023-12-13 21:47:00
스포티비뉴스
3670273.jpg
2023-12-14 07:24:00
헤럴드POP
640668.jpg
2023-12-14 07:35:00
텐아시아
3318823.jpg
2023-12-14 08:17:00
일간스포츠
4088878.jpg
2023-12-14 07:19:00
스포츠조선
1671855.jpg
2023-12-14 04:01:00
엑스포츠뉴스
3318817.jpg
2023-12-14 07:26:00
일간스포츠
4088866.jpg
2023-12-14 06:31:00
스포츠조선
1671861.jpg
2023-12-14 06:31:00
엑스포츠뉴스
4088865.jpg
2023-12-14 06:21:00
스포츠조선
3201574.jpg
2023-12-14 08:10:00
스타뉴스
4983462.jpg
2023-12-14 04:31:00
OSEN
4983459.jpg
2023-12-14 00:55:00
OSEN
12265730.jpg
2023-12-14 02:23:00
뉴시스
931684.jpg
2023-12-14 07:44:00
스포츠경향
4983427.jpg
2023-12-13 21:21:00
OSEN
3794870.jpg
2023-12-14 07:16:00
마이데일리
4983479.jpg
2023-12-14 06:55:00
OSEN
802775.jpg
2023-12-14 06:17:00
뉴스엔
802799.jpg
2023-12-14 07:09:00
뉴스엔
802766.jpg
2023-12-14 06:15:00
뉴스엔
12265640.jpg
2023-12-13 22:03:00
뉴시스
802722.jpg
2023-12-13 20:53:

In [ ]:
import pandas as pd

# posts 리스트를 DataFrame으로 변환
df = pd.DataFrame(posts, columns=['Date', 'News Name', 'Category', 'Image Path', 'News Title', 'Post Origin Text', 'Link'])

# CSV 파일로 저장
df.to_csv('scraped_data.csv', index=False, encoding='utf-8')


In [2]:
posts

[[datetime.datetime(2023, 12, 13, 19, 7),
  '뉴스1',
  'Entertain',
  'C:/Users/user/Downloads/news_image/entertain_image/7233319.jpg',
  '신동엽x이소라 "결혼했으면 2~3년 안에 이혼"…23년 만의 훈훈한 만남(종합)',
  '유튜브 채널 \'메리앤시그마\' 13일 공개\n이미지 원본보기\n유튜브 \'메리앤시그마\' 캡처\n\n(서울=뉴스1) 박하나 기자 = 방송인 신동엽, 이소라가 두 사람이 결혼했다면 이혼했을 거라고 입을 모았다.\n\n13일 오후 6시 유튜브 채널 \'메리앤시그마\'를 통해 공개된 \'신동엽 이소라 연애 타이밍 잘 맞았다면? | 이소라의 슈퍼마켙 소라 EP.01-2\' 영상에서는 전 연인인 이소라, 신동엽의 못다 한 이야기가 담겼다.\n\n이소라보다 먼저 \'짠한형 신동엽\'으로 유튜브에 도전한 신동엽은 "나야 하도 차에 술을 많이 갖고 다니고, 먹다가 마시다가 차에서 갖고 오고 이런 것 때문에 시작했는데"라며 유튜브의 남다른 파급력을 느끼고 있다고 운을 뗐다. 신동엽은 유튜브 이후 주위 반응을 보며 행복감을 느끼며 성공했다는 느낌이 들었다고.\n\n이소라는 동갑내기 친구 엄정화가 50세가 되자 "나이를 초월하자"며 말한 대로 하는 모습을 보고, 유튜브를 시작하고 용기를 얻었다고 고백했다. 신동엽은 "그런 점을 배울 필요가 있다"라며 공감했다.\n\n이후 이소라와 신동엽은 두 사람 모두와 절친한 방송인 서장훈에게 연락해 시선을 모았다. 서장훈은 전 연인인 이소라, 신동엽의 만남에 "두 분 다 대단하십니다, 시대를 앞서 나간다"라고 감탄했다. 그러자 이소라는 "너도 우리 앞서가는 시대에 동참시킬까"라고 너스레를 떨며 서장훈과의 만남을 약속했다.\n\n신동엽은 "만날 때 득 본 게 많아서, 헤어진 다음에도 너에게 부끄럽지 않은 사람이 되려고 했다"라고 소회를 밝혔다. 이어 신동엽은 "참 타이밍이 안 맞았다"라고 말하자 이소라가 "얼마나 

In [4]:
len(posts)

210

In [6]:
import psycopg2
from datetime import datetime
# 여러분의 데이터베이스 연결 정보
db_connection = psycopg2.connect(
    host="localhost",
    dbname='final_project',
    user='postgres',
    password="ilpg88**",
    port=5432
)
# 데이터베이스 커서 생성
cursor = db_connection.cursor()
for news_item in posts:
    try:
        cursor.execute("""
            INSERT INTO final_project.entertain_news
            (create_dt, news_name, news_genre, news_image_path, news_title, news_origin, news_url, news_summary)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (news_url) DO NOTHING;
        """, (news_item[0], news_item[1], news_item[2], news_item[3], news_item[4], news_item[5], news_item[-1], ''))
    except Exception as e:
        print(f"Error inserting news_item: {news_item}")
        print(f"Error message: {e}")
# 변경사항을 저장
db_connection.commit()
# 연결 종료
cursor.close()
db_connection.close()

In [12]:
import psycopg2
from datetime import datetime
# 여러분의 데이터베이스 연결 정보
db_connection = psycopg2.connect(
    host="localhost",
    dbname='final_project',
    user='postgres',
    password="ilpg88**",
    port=5432
)
# 데이터베이스 커서 생성
cursor = db_connection.cursor()
cursor.execute("""select * from final_project.entertain_news;""")

result = cursor.fetchall()

# 변경사항을 저장
db_connection.commit()
# 연결 종료
cursor.close()
db_connection.close()

In [13]:
df = pd.DataFrame(result)

In [14]:
df

,0,1,2,3,4,5,6,7,8
0,1,2023-12-13 17:19:00,OSEN,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"'고려거란전쟁' 이원종 ""김동준 연기력 논란, 오해 소지..완성된 황제 기대"" [인...",이미지 원본보기\n\n\n[OSEN=장우영 기자] 배우 이원종이 ‘고려 거란 전쟁’...,https://entertain.naver.com/ranking/read?oid=1...,
1,2,2023-12-13 16:54:00,스포티비뉴스,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"(여자)아이들 민니, 美공연 중 퇴장…큐브 ""컨디션 난조, 건강 최우선""[전문]",이미지 원본보기\n▲ (여자)아이들 민니. ⓒ곽혜미 기자\n\n\n[스포티비뉴스=정...,https://entertain.naver.com/ranking/read?oid=4...,
2,3,2023-12-13 15:47:00,스포츠조선,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"배우 주해미, 사망설 진짜였다..“치료 못 받고 떠나”",이미지 원본보기\n[스포츠조선닷컴 박아람 기자] 홍콩 스타 배우 주해미가 57세의 ...,https://entertain.naver.com/ranking/read?oid=0...,
3,4,2023-12-13 16:51:00,엑스포츠뉴스,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"""수사 마무리 단계""…지드래곤, '마약 의혹' 훌훌 털고 컴백까지 할까 [엑's 이슈]",이미지 원본보기\n\n\n(엑스포츠뉴스 명희숙 기자) 그룹 빅뱅 지드래곤(권지용) ...,https://entertain.naver.com/ranking/read?oid=3...,
4,5,2023-12-13 15:38:00,뉴스엔,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"엑소 세훈, 12월 21일 입대 “늦게 소식 전해 죄송”[전문]",이미지 원본보기\n엑소 세훈(사진=뉴스엔DB)\n\n\n[뉴스엔 이하나 기자] 엑소...,https://entertain.naver.com/ranking/read?oid=6...,
...,...,...,...,...,...,...,...,...,...
297,416,2023-12-05 07:33:00,마이데일리,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"'열애' 이장우, ♥여친 조혜원 누구길래? '나혼산' 하차 고민까지?→""결혼하고 싶...","이미지 원본보기\n배우 조혜원(왼쪽), 이장우 / 배우 조혜원, 한혁승 기자 han...",https://entertain.naver.com/ranking/read?oid=1...,
298,417,2023-12-05 07:02:00,스포츠조선,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"[SC초점] 한국인도, 한국어도 없다…글로벌 K팝, 어떻게 보시나요",이미지 원본보기\n[스포츠조선 백지은 기자] K팝에서 'K'가 사라지고 있다. 이제...,https://entertain.naver.com/ranking/read?oid=0...,
299,418,2023-12-05 18:58:00,스포츠서울,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,‘지각’ 한효주까지…‘어쩌다 사장3’ 언제까지 ‘어쩌다’ 장사할 거야? 김밥으로 생...,이미지 원본보기\ntvN ‘어쩌다 사장2’ 사진 | tvN\n\n[스포츠서울 | 유...,https://entertain.naver.com/ranking/read?oid=4...,
300,419,2023-12-05 08:21:00,엑스포츠뉴스,Entertain,C:/Users/user/Downloads/news_image/entertain_i...,"박진영 ""비♥김태희 두 딸, 끼 多…내 딸들과 걸그룹 계획"" (라스)",이미지 원본보기\n\n\n(엑스포츠뉴스 이창규 기자) JYP엔터테인먼트 수장이자 가...,https://entertain.naver.com/ranking/read?oid=3...,


In [15]:
df.to_csv('naver_entertain.csv', encoding='utf-8-sig')